# An Example of Applying Black-Litterman Model
*Xiaoqiong(Joan) Zhang*

Summary of the Black-Litterman Model

1. The equlibirum (excess) return vector $\pi = \delta_{eq}\Sigma w_{eq}$ from the CAPM pricing model, where the equlibrium risk aversion measure $\delta = \frac{R_{mkt}-R_f}{\sigma_{mkt}^2}$, and  is the equlibrium market portfolio.


2. The random variable $R$ is the $N *1$ security return vector, and , where  is the expected return vector (with prior distribution), and  is the known covariance matrix.
The expected return vector  has the prior distribution , where  measures the uncertainty on the prior, usually we set .
The portfolio manager has  active views, which can be written as , where  is the view portfolio matrix,  is the view expected return vector, and , and .
The posterior distribution of the expected return vector  given active views  is , where, and .
The distribution of the security return vector  conditional on actie views  has been updated to .
The optimal weight vector  given a risk aversion measure : .

In [1]:
# Load monthly return information for asset class in CAPM model

In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [3]:
data = pd.read_excel('CAPM_AssetClasses.xls','MonthlyReturns',header=0,skiprows=1,parse_date=True,index_col=[0])
data = data.to_period('M')
data.index = data.index.rename('Period')

num_month = data.shape[0]
ac_name = data.columns
num_ac = len(ac_name)

In [4]:
data.head()

,Bloomberg Barclays - U.S. TIPS Index,Bloomberg Barclays - U.S. Aggregate Index,BofA Merrill Lynch - U.S. High Yield Index,FTSE - Non U.S. Govt Bond Index ($),Russell - 1000 Growth Index,Russell - 1000 Value Index,Russell - 2000 Growth Index,Russell - 2000 Value Index,MSCI - EAFE Index ($Net),MSCI - Emerging Markets Index ($ Net),S&P - GSCI Total Index,MSCI - U.S. REIT Index
Period,,,,,,,,,,,,
1999-01,1.164369,0.71,1.35,-1.57,5.87,0.80,4.50,-2.27,-0.30,-1.507453,0.440000,-2.685540
1999-02,-0.698657,-1.75,-0.68,-3.51,-4.57,-1.41,-9.15,-6.83,-2.38,0.963531,-4.600000,-1.644916
1999-03,-0.014495,0.55,1.16,0.19,5.27,2.07,3.56,-0.82,4.17,13.214558,16.879999,-0.545957
1999-04,0.661948,0.32,1.83,-0.15,0.13,9.34,8.83,9.13,4.05,12.370000,4.130000,9.669238
1999-05,0.688692,-0.88,-0.92,-2.02,-3.07,-1.10,0.16,3.07,-5.15,-0.580000,-5.460000,2.119436


In [5]:
# Load market portfolio from excel
mkt = pd.read_excel('CAPM_AssetClasses.xls','MarketPortfolio',header=1,rows=2)
w_mkt = mkt.iloc[0]
rp_mkt = mkt.values[2][0]*100
print('msrket risk premium (per annum) is ',rp_mkt,'%')

msrket risk premium (per annum) is  5.0 %


In [6]:
w_mkt

Bloomberg Barclays - U.S. TIPS Index          0.040
Bloomberg Barclays - U.S. Aggregate Index     0.292
BofA Merrill Lynch - U.S. High Yield Index    0.030
FTSE - Non U.S. Govt Bond Index ($)           0.038
Russell - 1000 Growth Index                   0.166
Russell - 1000 Value Index                    0.166
Russell - 2000 Growth Index                   0.024
Russell - 2000 Value Index                    0.024
MSCI - EAFE Index ($Net)                      0.120
MSCI - Emerging Markets Index ($ Net)         0.045
S&P - GSCI Total Index                        0.034
MSCI - U.S. REIT Index                        0.021
Name: Weight, dtype: float64

**Using historical returns to calculate the covariance matrix**

Note: need to annualize the covariance matrix because we use monthly returns


In [7]:
Sigma = data.cov()*12
round(Sigma,2)

,Bloomberg Barclays - U.S. TIPS Index,Bloomberg Barclays - U.S. Aggregate Index,BofA Merrill Lynch - U.S. High Yield Index,FTSE - Non U.S. Govt Bond Index ($),Russell - 1000 Growth Index,Russell - 1000 Value Index,Russell - 2000 Growth Index,Russell - 2000 Value Index,MSCI - EAFE Index ($Net),MSCI - Emerging Markets Index ($ Net),S&P - GSCI Total Index,MSCI - U.S. REIT Index
Bloomberg Barclays - U.S. TIPS Index,32.10,14.93,15.22,24.36,1.84,3.13,-3.35,1.68,11.30,21.15,27.05,29.40
Bloomberg Barclays - U.S. Aggregate Index,14.93,11.43,5.62,15.38,-5.02,-3.76,-9.13,-4.94,0.97,2.15,-3.13,13.50
BofA Merrill Lynch - U.S. High Yield Index,15.22,5.62,79.82,13.88,90.12,80.88,120.25,99.12,99.64,134.48,65.84,110.21
FTSE - Non U.S. Govt Bond Index ($),24.36,15.38,13.88,65.51,8.89,15.50,2.54,10.01,43.68,44.61,36.94,41.42
Russell - 1000 Growth Index,1.84,-5.02,90.12,8.89,277.28,188.59,315.27,205.29,219.13,269.40,106.05,162.56
Russell - 1000 Value Index,3.13,-3.76,80.88,15.50,188.59,211.50,221.04,219.40,192.20,220.14,95.27,197.93
Russell - 2000 Growth Index,-3.35,-9.13,120.25,2.54,315.27,221.04,496.54,333.67,267.42,345.33,149.62,236.67
Russell - 2000 Value Index,1.68,-4.94,99.12,10.01,205.29,219.40,333.67,320.15,209.11,249.70,111.21,268.81
MSCI - EAFE Index ($Net),11.30,0.97,99.64,43.68,219.13,192.20,267.42,209.11,266.34,302.58,155.21,190.11
MSCI - Emerging Markets Index ($ Net),21.15,2.15,134.48,44.61,269.40,220.14,345.33,249.70,302.58,475.74,208.16,222.84


**Calculate the market risk aversion coefficient, i.e.,** the equilibrium/mkt risk aversion measure, which equals the market rsik premium over the market portfolio's variance level, therefore, its unit is return per unit of variance.

Note: We assume risk-free return equals to 0

In [8]:
delta_eq = rp_mkt/(np.dot(w_mkt.T,np.dot(Sigma,w_mkt)))
print('Variance of The Market Portfolio = %.2f. \n' %  np.dot(w_mkt.T,np.dot(Sigma,w_mkt)))
print('The risk aversion coefficient = %.4f. \n' % delta_eq)


Variance of The Market Portfolio = 84.14. 

The risk aversion coefficient = 0.0594. 



**Calculate the equalibrium return vector**

In [9]:
Pi = np.dot(delta_eq,np.dot(Sigma,w_mkt)).reshape(-1,1)

for i in range(len(data.columns)):
    print('%d:%s - %.2f%%' % (i,data.columns[i],Pi[i]))

0:Bloomberg Barclays - U.S. TIPS Index - 0.69%
1:Bloomberg Barclays - U.S. Aggregate Index - 0.20%
2:BofA Merrill Lynch - U.S. High Yield Index - 3.65%
3:FTSE - Non U.S. Govt Bond Index ($) - 1.31%
4:Russell - 1000 Growth Index - 8.14%
5:Russell - 1000 Value Index - 7.10%
6:Russell - 2000 Growth Index - 9.96%
7:Russell - 2000 Value Index - 7.96%
8:MSCI - EAFE Index ($Net) - 8.32%
9:MSCI - Emerging Markets Index ($ Net) - 10.23%
10:S&P - GSCI Total Index - 5.34%
11:MSCI - U.S. REIT Index - 7.50%


**Check** if use the equlibrium return vector in the CAPM model, we'll get the market portfolio as the optimal solution

In [10]:
w = np.dot(np.linalg.inv(Sigma),Pi)/sum(np.dot(np.linalg.inv(Sigma),Pi))
for i in range(len(w_mkt)):
    print('%d.%s: %.2f%% - %.2f%%' % (i,w_mkt.index[i],w_mkt[i],w[i]))

0.Bloomberg Barclays - U.S. TIPS Index: 0.04% - 0.04%
1.Bloomberg Barclays - U.S. Aggregate Index: 0.29% - 0.29%
2.BofA Merrill Lynch - U.S. High Yield Index: 0.03% - 0.03%
3.FTSE - Non U.S. Govt Bond Index ($): 0.04% - 0.04%
4.Russell - 1000 Growth Index: 0.17% - 0.17%
5.Russell - 1000 Value Index: 0.17% - 0.17%
6.Russell - 2000 Growth Index: 0.02% - 0.02%
7.Russell - 2000 Value Index: 0.02% - 0.02%
8.MSCI - EAFE Index ($Net): 0.12% - 0.12%
9.MSCI - Emerging Markets Index ($ Net): 0.04% - 0.05%
10.S&P - GSCI Total Index: 0.03% - 0.03%
11.MSCI - U.S. REIT Index: 0.02% - 0.02%


**Prior Variance $C $of Expected Return $\theta$**

The expected return vector $\theta$ is assumed to follow a prior distribution $N(\Pi,C)$. 

In the BL model, the prior variance $C$is set to $\tau\Sigma$, i.e., proportional to security fundamental covariance matrix. 

The choice of $\tau$ is one of the major roadblocks in practical application of the BL model (another one is the choice of the view uncertain matrix $\Omega$). 

Since uncertainties about the exepcted return vector $\theta$ is less than variabilities of returns themselves, $\tau$ is usually set to a value less than 1. 

Here are some options  in specifying the value of $\tau$:
1. Black and Litterman (1992) -  $\tau$ to be close to 0, typically 0.01 to 0.05.
2. Satchell and Scowcroft (2000) - $\tau=1$.
3. Blamont and Firoozye (2003) - $\tau=1/T$, where $T$ is the number of time periods in observations. 

In this example, let's use method 3.

In [11]:
# The number of years of observations in the data set
T = num_month/12

# Set tau to 1/T
tau = 1/T

**Active Views:**
The portfolio manager of the asset allocation portfolio, you have the following three active views for the next year:
* View 1: *U.S. High Yield Corp Bonds* will return 5%;
* View 2: *Emerging Markets Equities(EM)* will outperform *Developed Non-US Markets Equities (EAFE)* by 4%;
* View 3: *U.S. Value Stocks* will outperfrom *U.S. Growth Stocks* by 3%.

View 1: *U.S. High Yield Corp Bonds* will return 5%

View 1 is an absolute view. The view portfolio has 100% allocation to *U.S. High Yield Corp Bonds*

In [12]:
# Locate the U.S. High Yield Corp Bonds
tmp_pos = 2;
print('---View 1--- \n (Long Leg) %s.\n' % ac_name[tmp_pos])

---View 1--- 
 (Long Leg) BofA Merrill Lynch - U.S. High Yield Index.



In [13]:
# Build the view portfolio corresponding to View 1
port_v1 = np.zeros(num_ac).reshape(1,-1)
port_v1[:,2] = 1

In [14]:
# The view portfolio's expected return is 5%
q1 = 5

In [15]:
# Compared to equilibrium return of the view portfolio
print('View 1: equlibrium return =%.2f%% active view return = %.2f%%'% (np.dot(port_v1,Pi),q1))

View 1: equlibrium return =3.65% active view return = 5.00%


This active view forecasts that the asset class performs better than what's implied by its equlibrium return, therefore, the corresponding BL portfolio tends to assign more weight to this asset class relative to the market portfolio.

View 2: *Emerging Markets Equities (EM)* will outperform *Developed Non-US  Markets Equities (EAFE)* by 4%

View 2 is an relative view. The view portfolio consists of a long leg which has 100% allocation to _EM Equities,_ and a short leg which has 100% allocation to _EAFE Equities._

In [16]:
# locate the EM Equities and EAFE equities
tmp_long = 9 #for the long leg
tmp_short = 8 # for the short leg
print('---View 2---\n (long Leg) %s. \n (Shore Leg) %s. \n'% (ac_name[tmp_long],ac_name[tmp_short]))

---View 2---
 (long Leg) MSCI - Emerging Markets Index ($ Net). 
 (Shore Leg) MSCI - EAFE Index ($Net). 



In [17]:
# build the view portfolio corresponding to View 2
port_v2 = np.zeros(num_ac).reshape(1,-1)
port_v2[:,tmp_long]=1
port_v2[:,tmp_short]=-1
port_v2

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.]])

In [18]:
# the view portfolio's expected return
q2 = 4

In [19]:
# compared to equlibrium return of the view portfolio
print('View 2: equlibrium return=%.2f%%, active view return=%.2f%%.\n' % (np.dot(port_v2,Pi),q2))

View 2: equlibrium return=1.92%, active view return=4.00%.



This active view forecasts that the performance difference between the 
long leg and short leg would be greater than what's implied by their equlibrium return difference, therefore, the Black-Litterman portfolio tends to allocate more weight to the long leg and less weight to the short leg relative to the market portfolio.

View 3: _U.S. Value Stocks_ will outperfrom _U.S. Growth Stocks_ by 3%

View 3 is another relative view.  However, this time both the long and short legs of the view portfolio themselves are sub-portfolio of multiple asset classes. The long leg is a sub-portfolio of _U.S. Large Cap Value_ and _U.S. Small Cap Value_; and the short leg is a sub-portfolio of _U.S. Large Cap Growth_ and _U.S. Small Cap Growth._ For each sub-portfolio, constituent asset classes are assigned with allocations proportional to their weights in the market portfolio, and their total alloctions sum up to 100%. (Note: another way to construct sub-portfolio is simply equal-weighting constituent asset classes.)

In [20]:
# locate the US Large Cap Value and U.S. Small Cap Value
tmp_long=np.array([5,7])#  for the long leg
tmp_short=np.array([4,6])#  for the short leg
print('---View 3---\n(Long Leg) 1)%s; 2)%s.\n(Short Leg) 1)%s; 2)%s.\n'% (ac_name[tmp_long[0]],ac_name[tmp_long[1]],ac_name[tmp_short[0]],ac_name[tmp_short[1]]))

---View 3---
(Long Leg) 1)Russell - 1000 Value Index; 2)Russell - 2000 Value Index.
(Short Leg) 1)Russell - 1000 Growth Index; 2)Russell - 2000 Growth Index.



In [21]:
# build the view portfolio corresponding to View 3
port_v3_long=np.zeros(num_ac).reshape(1,-1)
port_v3_short=np.zeros(num_ac).reshape(1,-1)
w = w_mkt.values.reshape(1,-1)

# sub-portfolio for the long leg
port_v3_long[:,tmp_long]=w[:,tmp_long]
port_v3_long=port_v3_long/np.sum(port_v3_long)

#sub-portfolio for the short leg
port_v3_short[:,tmp_short]=w[:,tmp_short]
port_v3_short=port_v3_short/np.sum(port_v3_short)

#view portfolio
port_v3=port_v3_long-port_v3_short

In [22]:
# the view portfolio's expected return
q3 = 3

In [23]:
# compared to equlibrium return of the view portfolio
print('View 3: equlibrium return=%.2f%%, active view return=%.2f%%.\n' % ((np.dot(port_v3,Pi),q3)))

View 3: equlibrium return=-1.16%, active view return=3.00%.



This active view forecasts that the performance difference between the long leg and short leg would be greater than what's implied by their equlibrium return difference, therefore, the Black-Litterman portfolio tends to allocate more weight to the long leg sub-portfolio and less weight to the short leg sub-portfolio relative to the market portfolio.

**View Matrix $P$ and Vew Return Vector $Q$ Given The Above Three Active Views**

The view matrix is a kxN matrix where each row corresponds to a view portfolio

In [24]:
P = np.vstack((port_v1,port_v2,port_v3))

pd.DataFrame(P.T,index=data.columns,columns=['View1','View2','View3'])

,View1,View2,View3
Bloomberg Barclays - U.S. TIPS Index,0.0,0.0,0.000000
Bloomberg Barclays - U.S. Aggregate Index,0.0,0.0,0.000000
BofA Merrill Lynch - U.S. High Yield Index,1.0,0.0,0.000000
FTSE - Non U.S. Govt Bond Index ($),0.0,0.0,0.000000
Russell - 1000 Growth Index,0.0,0.0,-0.873684
Russell - 1000 Value Index,0.0,0.0,0.873684
Russell - 2000 Growth Index,0.0,0.0,-0.126316
Russell - 2000 Value Index,0.0,0.0,0.126316
MSCI - EAFE Index ($Net),0.0,-1.0,0.000000
MSCI - Emerging Markets Index ($ Net),0.0,1.0,0.000000


In [25]:
# View Return Vector Q
Q = np.vstack((q1,q2,q3))

**View Uncertainty Matrix $\Omega$**

In the BL model, $\Omega$ is a diagonal matrix containing diagonal elements $\omega_i$ ($i=1,...,k$) which measures the portfolio manager's uncertainty on view $k$. The uncertainty matrix $\Omega$ reflects how confident the portfolio manager is on his/her active views subjectively. There are different ways to specify $\Omega$, and we introduce two methods:

* Make statistical assumption. For example, a portfolio manager has View A return of 6%, and the projection of the return would fall between 5% and 7% with 95% confidence level, which implicates a two sigma of 1%=(7%-5%)/2, or, one standard deviation of 0.5%. 

* He and Litterman (1999) - $\omega_k=\tau\cdot (p_k\Sigma p_k')$, i.e., the confidence of a view equals $\tau$ (from the prior distribution of the expected return vector $\theta$) times the variance of the view portfolio ($p_k\Sigma p_k'$). (note: using this method, then $\tau$ has no impact on the value of the updated expected reutrn vector $\nu$).

Let's use method 2 in this example.

In [26]:
# calculate the first diagonal element (corresponding to View 1) 
omega1 = tau*np.dot(port_v1,np.dot(Sigma,port_v1.T))
omega2 = tau*np.dot(port_v2,np.dot(Sigma,port_v2.T))
omega3 = tau*np.dot(port_v3,np.dot(Sigma,port_v3.T))

In [27]:
# the View uncertainty matrix
Omega = np.diag(np.hstack((omega1,omega2,omega3))[0])

**Black-Litterman Results**

The distribution of the security return vector $R$ conditional on actie views $Q$ has been updated to $N(\nu,H^{-1}+\Sigma)$, where:

* $H=(P'\Omega^{-1}P+C^{-1})$
* $\nu=H^{-1}(P'\Omega^{-1}Q+C^{-1}\Pi)$


And the optimal weight vector $w^*$ given a risk aversion measure $\delta$: $w^*=\delta^{-1}(H^{-1}+\Sigma)^{-1}\nu$.

Let's first calculate $H$ and updated expected return vector $\nu$:

In [28]:
# calculate C
C = tau*Sigma

# calculate H
H = np.dot(P.T,np.dot(np.linalg.inv(Omega),P)) + np.linalg.inv(C)

# calculate nu

nu = np.dot(np.linalg.inv(H),(np.dot(P.T,np.dot(np.linalg.inv(Omega),Q))+np.dot(np.linalg.inv(C),Pi).reshape(-1,1)))
            

In [29]:
# let's compare the equlibrium return Pi and the updated expected return nu
pd.DataFrame(np.hstack((Pi,nu)),index=ac_name,columns=['Exp_Ret','Updated_Exp_Ret'])

,Exp_Ret,Updated_Exp_Ret
Bloomberg Barclays - U.S. TIPS Index,0.692761,0.931737
Bloomberg Barclays - U.S. Aggregate Index,0.195100,0.282547
BofA Merrill Lynch - U.S. High Yield Index,3.647922,4.362287
FTSE - Non U.S. Govt Bond Index ($),1.313067,1.565618
Russell - 1000 Growth Index,8.136533,7.452521
Russell - 1000 Value Index,7.095385,8.370254
Russell - 2000 Growth Index,9.956679,9.514337
Russell - 2000 Value Index,7.958892,9.301724
MSCI - EAFE Index ($Net),8.316822,8.798851
MSCI - Emerging Markets Index ($ Net),10.234905,11.655372


Recall the three active views:

* View 1: _U.S. High Yield Corp Bonds _will return 5%;
* View 2:_ Emerging Markets Equities (EM)_ will outperform _Developed Non-US 
% Markets Equities (EAFE) _by 4%; 
* View 3: _U.S. Value Stocks_ will outperfrom _U.S. Growth Stocks _by 3%. 

We already assumed a risk aversion $\delta_{eq}$ (see the section of _Reverse Opitmization_), which will be used to calculate the optimal allocation solution from the Black Litterman model: $w^*=\delta^{-1}(H^{-1}+\Sigma)^{-1}\nu$.


In [30]:
# The optimal weight vector from BL model
w_opt = np.dot(delta_eq**(-1)*np.linalg.inv(np.linalg.inv(H)+Sigma),nu)

In [31]:
w_opt

array([[ 0.03809524],
       [ 0.27809524],
       [ 0.17696649],
       [ 0.03619048],
       [-0.12512684],
       [ 0.44131732],
       [-0.01809063],
       [ 0.06380491],
       [-0.02555876],
       [ 0.18270162],
       [ 0.03238095],
       [ 0.02      ]])

In [32]:
# Check if net leverage of 1
print('Net Leverage of the BL Optimal Portfolio = %.1f.\n' % np.sum(w_opt))

Net Leverage of the BL Optimal Portfolio = 1.1.



_Note:the BL optimal portfolio may have non-unity net leverage._

In [33]:
# let's compare with the equlibrium portfolio, i.e., 
# the market portfolio which is the optimal one without active views

pd.DataFrame(np.hstack((w.T,nu)),columns=['Mkt_Port','BL_port'],index=ac_name)

,Mkt_Port,BL_port
Bloomberg Barclays - U.S. TIPS Index,0.040,0.931737
Bloomberg Barclays - U.S. Aggregate Index,0.292,0.282547
BofA Merrill Lynch - U.S. High Yield Index,0.030,4.362287
FTSE - Non U.S. Govt Bond Index ($),0.038,1.565618
Russell - 1000 Growth Index,0.166,7.452521
Russell - 1000 Value Index,0.166,8.370254
Russell - 2000 Growth Index,0.024,9.514337
Russell - 2000 Value Index,0.024,9.301724
MSCI - EAFE Index ($Net),0.120,8.798851
MSCI - Emerging Markets Index ($ Net),0.045,11.655372


As expected and aligned with the three active views, the resulted portfolio  overweigts High Yield, Emerging Market Equities, U.S. Equities Value, and underweights Developed Non-US Equities, U.S. Equities Growth. We also see that every asset class's allocation has changed even though it may not be directly related to any one of the three active views. It is because the Black Litterman model take into account the correlations among all asset classes, which leads to a more accurately titled portfolio from a holistic point of view on portfolio construction.